In [1]:
import pandas as pd
import numpy as np
import json
import re # Regular expresions

from langdetect import detect

# Read data

In [14]:
# Open and decode the files
whayner_file = open('../scraper-data-whayner/data.json', encoding="utf8")
whayner_data = json.load(whayner_file)
whayner_dataframe = pd.DataFrame(whayner_data['videos'])

silvia_file = open('../scraper-data-silvia/data.json', encoding="utf8")
silvia_data = json.load(silvia_file)
silvia_dataframe = pd.DataFrame(silvia_data['videos'])

estefany_file = open('../scraper-data-estefany/data.json', encoding="utf8")
estefany_data = json.load(estefany_file)
estefany_dataframe = pd.DataFrame(estefany_data['videos'])

pedro_gomez_file = open('../scraper-data-pedrofelipe/data.json', encoding="utf8")
pedro_gomez_data = json.load(pedro_gomez_file)
pedro_gomez_dataframe = pd.DataFrame(pedro_gomez_data['videos'])

andres_file = open('../scraper-data-andres/data.json', encoding="utf8")
andres_data = json.load(andres_file)
andres_dataframe = pd.DataFrame(andres_data['videos'])

pedro_chaparro_file = open('../scraper-data-pedroandres/data.json', encoding="utf8")
pedro_chaparro_data = json.load(pedro_chaparro_file)
pedro_chaparro_dataframe = pd.DataFrame(pedro_chaparro_data['videos'])

# Concatenate in a single dataframe
complete_dataframe = pd.concat([whayner_dataframe, 
                               silvia_dataframe, 
                               estefany_dataframe, 
                               pedro_gomez_dataframe, 
                               andres_dataframe,
                               pedro_chaparro_dataframe])

# Reset index
complete_dataframe = complete_dataframe.reset_index(drop=True)


# print(complete_dataframe.shape)

In [15]:
# complete_dataframe.head()

In [16]:
# complete_dataframe.tail()

In [17]:
complete_dataframe.to_json('./initial.json', force_ascii=False, orient='table', index=False)

# Convert to lowercase letters

In [18]:
for dataframe_index in complete_dataframe.index: 
    title = complete_dataframe['title'][dataframe_index]
    description = complete_dataframe['description'][dataframe_index]
    tags = complete_dataframe['tags'][dataframe_index]
    
    complete_dataframe['title'][dataframe_index]= title.lower()
    complete_dataframe['description'][dataframe_index] = description.lower()
    complete_dataframe['tags'][dataframe_index] = tags.lower()

# Remove links 

In [19]:
for dataframe_index in complete_dataframe.index: 
    title = complete_dataframe['title'][dataframe_index]
    description = complete_dataframe['description'][dataframe_index]
    # Replace strings that starting with http or www with ''
    title = re.sub(r'http\S+', '', title, flags=re.MULTILINE)
    description = re.sub(r'http\S+', '', description, flags=re.MULTILINE) 
    complete_dataframe['title'][dataframe_index] =  re.sub(r'www\S+', '', title, flags=re.MULTILINE)
    complete_dataframe['description'][dataframe_index] = re.sub(r'www\S+', '', description, flags=re.MULTILINE) 

# Remove unwanted characters (All that is not alfanum)

In [20]:
for dataframe_index in complete_dataframe.index: 
    title = complete_dataframe['title'][dataframe_index]
    description = complete_dataframe['description'][dataframe_index]
    tags = complete_dataframe['tags'][dataframe_index]

    # Remove not alfanumeric chars (with exceptions)
    new_title = re.sub(r'[^a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ ]+', '', title)
    new_description = re.sub(r'[^a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ ]+', '', description)
    new_tags = re.sub(r'[^a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ, ]+', '', tags)
    
    # Remove redundant spaces
    new_title = re.sub(' +', ' ', new_title).strip()
    new_description = re.sub(' +', ' ', new_description).strip()
    new_tags = re.sub(' +', ' ', new_tags).strip()

    # Replace
    complete_dataframe['title'][dataframe_index] = new_title
    complete_dataframe['description'][dataframe_index] = new_description
    complete_dataframe['tags'][dataframe_index] = new_tags

# Remove entries whose language is not english or spanish

In [21]:
accepted_languages = ['es', 'en', 'it']
removed = []

# Create a copy of original dataframe
df = complete_dataframe.copy()

for dataframe_index in complete_dataframe.index: 
    video_texts = complete_dataframe['description'][dataframe_index] + complete_dataframe['title'][dataframe_index] + complete_dataframe['tags'][dataframe_index]
    language = detect(video_texts)
    
    if(language not in accepted_languages):
        # print(language)
        # print(complete_dataframe['url'][dataframe_index])
        removed.append({'lang': language, 'url': complete_dataframe['url'][dataframe_index]})
        df = df.drop(dataframe_index)

print(removed)

[{'lang': 'nl', 'url': 'https://www.youtube.com/watch?v=Q7aJYn94YQc'}, {'lang': 'af', 'url': 'https://www.youtube.com/watch?v=mnThZdxCTrw'}, {'lang': 'nl', 'url': 'https://www.youtube.com/watch?v=xshpILh3cLs'}, {'lang': 'ca', 'url': 'https://www.youtube.com/watch?v=VFN3zJTQwkc'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=SuD0nIcRkKY'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=H_Zt3Vy6Suc'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=elUJCEC06r8'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=7R1VUePOqxE'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=ILPex8f-CYM'}, {'lang': 'ca', 'url': 'https://www.youtube.com/watch?v=jiIB_QEBMfc'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=BWRNNXa-S3Y'}, {'lang': 'da', 'url': 'https://www.youtube.com/watch?v=LFW6iq74WpA'}, {'lang': 'fr', 'url': 'https://www.youtube.com/watch?v=0b-keN3GEO4'}, {'lang': 'fr', 'url': 'https://www.youtube.com/watch?v=SBXoQH1ufmQ'}, {'lang': 'fr', 'url

In [22]:
complete_dataframe.shape

(12849, 5)

In [28]:
df.shape

(12621, 5)

# Save as json

In [24]:
df.to_json('./data.json', force_ascii=False, orient='table', index=False)

# Counting null values - Deleting empty titles

In [2]:
data = open('./data.json', encoding="utf8")
# show all rows
#pd.set_option('display.max_rows', None)

temp_df = pd.read_json(data)
df = pd.json_normalize(temp_df['data'])

# find indexes where there is just an empty string
indexes = df[df['title'] == ''].index.tolist()
# Counting null data in each column
print(df.isnull().sum())

update_df = df.copy()
print("shape before drop:", update_df.shape)
print("indexes with empty titles:", indexes)
update_df = update_df.drop(index = indexes)
print("original shape:", df.shape, "updated shape:", update_df.shape)

#checking if there is another empty title in the updated
print(len(update_df[update_df['title'] == ''].index))

# reseting the index
update_df = update_df.reset_index(drop=True)

url            0
title          0
description    0
tags           0
thumbnail      0
dtype: int64
shape before drop: (12621, 5)
indexes with empty titles: [688, 713, 768, 7782, 11967, 11988, 12013, 12038, 12085]
original shape: (12621, 5) updated shape: (12612, 5)
0


# Deleting repeated words in tags 

In [3]:
for index in update_df['tags'].index:
    # take row, separate by ',' and remove space (strip) before and after each (map) word
    original = map(str.strip, update_df['tags'][index].split(','))
    unique_words = set(original)
    if '' in unique_words:
        unique_words.remove('')
    # join words in unique_words, leaving an space between them
    update_df['tags'][index] = ' '.join(unique_words)

# Deleting videos with tag:
"teléfono con cámara video subir gratuito teléfono con video compartir"
<br> Be carefull! if the script is executed again, this repeated tag might be change 

In [6]:
duplicate_tags = update_df[update_df['tags'] == 'teléfono con cámara video subir gratuito teléfono con video compartir'].index.tolist()
print("original shape:", update_df.shape)
update_df = update_df.drop(index = duplicate_tags)
print("updated shape:", update_df.shape)
# reseting the index
update_df = update_df.reset_index(drop=True)

original shape: (12612, 5)
updated shape: (10634, 5)


# Save as json

In [7]:
update_df.to_json('./data.json', force_ascii=False, orient='table', index=False)